In [5]:
import sys
import os
current_dir = os.getcwd()
src_path = os.path.join(current_dir, '..', 'src')
if os.path.exists(src_path):
    sys.path.append(src_path)
else:
    # Try alternative path structure
    alt_src_path = os.path.join(current_dir, 'src')
    if os.path.exists(alt_src_path):
        sys.path.append(alt_src_path)
    else:
        print("Warning: Could not find src directory. Please ensure the path is correct.")



# RAG Pipeline Demo

This notebook demonstrates the Retrieval-Augmented Generation (RAG) pipeline for analyzing financial complaints. The pipeline combines:

1. **Retrieval**: Uses FAISS vector search to find relevant complaint chunks
2. **Generation**: Leverages a language model (flan-t5-base) to generate answers
3. **Context Integration**: Combines retrieved context with user questions

## What's Next

The following cell will:
- Load the pre-built embedding indexer with FAISS vectors
- Initialize the RAG pipeline with the language model
- Test the complete pipeline with a sample question
- Display the generated answer and retrieved source chunks

This demonstrates the end-to-end functionality of our complaint analysis system.


In [7]:
from embedding_indexing import create_embedding_indexer
from rag_pipeline import RAGPipeline
# Load the indexer
indexer = create_embedding_indexer()
indexer.load_faiss_index(os.path.join(current_dir, "..", "vector_store"))

# Initialize the RAG pipeline
rag = RAGPipeline(indexer=indexer, llm_model="google/flan-t5-base", device=-1, top_k=5)

# Example question
question = "What are common issues with credit card complaints?"
result = rag.answer_question(question)

print("Question:", result['question'])
print("Generated Answer:", result['answer'])
print("Top Retrieved Sources:")
for src in result['retrieved_sources']:
    print("-", src['text'][:50], "...")




model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Question: What are common issues with credit card complaints?
Generated Answer: a deceptive and inconsistent interest charge on one of my Citi credit cards, despite paying the full statement balance during the billing cycle and having a ---
Top Retrieved Sources:
- is the credit card department, not the trouble dep ...
- I am submitting this complaint regarding a decepti ...


## Testing the RAG Pipeline 
**The next cell will thoroughly test our RAG pipeline with multiple scenarios:**
# 
1. **Comprehensive Question Testing**
**We'll test the pipeline with 5 different types of questions covering various aspects of financial complaints:**
 - General complaint patterns
 - Resolution processes  
 - Technology-related issues
 - Performance metrics
 - Security concerns
# 
2. **Component-Level Testing**
**We'll break down the pipeline to test each component individually:**
 - **Retrieval**: Test the FAISS vector search functionality
 - **Prompt Engineering**: Verify the context integration
 - **Generation**: Test the LLM response generation
# 
3. **Performance Evaluation**
**We'll measure the end-to-end processing time to assess system performance.**
# 
- This testing approach helps us understand both the overall pipeline performance and identify any bottlenecks in individual components.


In [22]:
# Test the RAG pipeline with different types of questions
test_questions = [
    "What are the most common customer complaints?",
    "How do customers typically resolve billing disputes?",
    "What issues do customers face with mobile banking?",
    "What are the typical resolution times for complaints?",
    "How do customers report fraud or suspicious activity?"
]

print("=== RAG Pipeline Evaluation ===\n")

for i, question in enumerate(test_questions, 1):
    print(f"Question {i}: {question}")
    result = rag.answer_question(question)
    print(f"Answer: {result['answer']}")
    print(f"Retrieved Sources: {len(result['retrieved_sources'])} chunks")
    print("-" * 80 + "\n")

# Test individual pipeline components
print("=== Testing Individual Components ===\n")

# Test retrieval only
test_question = "What are common credit card issues?"
retrieved_chunks = rag.retrieve(test_question)
print(f"Retrieved {len(retrieved_chunks)} chunks for: '{test_question}'")
for i, chunk in enumerate(retrieved_chunks[:2], 1):
    print(f"Chunk {i}: {chunk['text'][:150]}...")

# Test prompt building
prompt = rag.build_prompt(test_question, retrieved_chunks)
print(f"\nGenerated prompt length: {len(prompt)} characters")
print(f"Prompt preview: {prompt[:200]}...")

# Test generation only
generated_answer = rag.generate(prompt)
print(f"\nGenerated answer: {generated_answer}")

# Performance evaluation
print("\n=== Performance Metrics ===")
import time

start_time = time.time()
result = rag.answer_question("What are common customer complaints?")
end_time = time.time()

print(f"Total processing time: {end_time - start_time:.2f} seconds")
print(f"Retrieved chunks: {len(result['retrieved_sources'])}")
print(f"Answer length: {len(result['answer'])} characters")


=== RAG Pipeline Evaluation ===

Question 1: What are the most common customer complaints?


Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer: lack of growth, lack of transparency, and lack of basic customer appreciation
Retrieved Sources: 2 chunks
--------------------------------------------------------------------------------

Question 2: How do customers typically resolve billing disputes?


Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer: They file a dispute on XX/XX/XXXX, after learning the attorney was disbarred, never provided any billing records, and failed to deliver the services promised ---
Retrieved Sources: 2 chunks
--------------------------------------------------------------------------------

Question 3: What issues do customers face with mobile banking?


...

Answer: a fraud alert on it without first resolving the fraud alert
Retrieved Sources: 2 chunks
--------------------------------------------------------------------------------

=== Testing Individual Components ===

Retrieved 5 chunks for: 'What are common credit card issues?'
Chunk 1: is the credit card department, not the trouble department ''...
Chunk 2: . Over the next 4 years and 6 months, I maintained a very good to excellent credit score, never missed a payment, and consistently used both cards res...

Generated prompt length: 1114 characters
Prompt preview: You are a financial analyst assistant for CrediTrust. Your task is to answer questions about customer complaints. Use the following retrieved complaint excerpts to formulate your answer. If the contex...


Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Generated answer: Credit card companies and banks have enormous power and consumers are left with situations like this. They should be REQUIRED to let you know at the Point of Sale what the problem is.

=== Performance Metrics ===


Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Total processing time: 4.75 seconds
Retrieved chunks: 2
Answer length: 77 characters


In [24]:
# Run qualitative evaluation
results = RAGPipeline.qualitative_evaluation(rag_pipeline, questions)

# Generate evaluation table
evaluation_table = RAGPipeline.evaluation_table(results)

# Display the evaluation table
display(Markdown(evaluation_table))


Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


| Question | Generated Answer | Retrieved Sources | Quality Score (1-5) | Comments/Analysis |
|---|---|---|---|---|
| What are the common issues customers face? | They are not satisfied with the service they receive. |  |  |  |
| How can I resolve a complaint about delayed service? | Ask for a refund. |  |  |  |